# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [ ]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = set(a)
    ret.update(set(b))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for transaction in self.data.values():
            if itemset.issubset(transaction):
                support += 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        frequent_itemsets = [] # list to contain freq ietmsets
        
        for each in list(tree.keys()):
            # check if support meets the minsup or not
            if search_space[each]['support']<self.minSup:
                search_space[each]["pruned"] = True 
            else: 
                frequent_itemsets.append(sorted(search_space[each]["itemset"]))
                search_space[each]["pruned"] = False

        # add freq itemsets into L[k]
        self.L.setdefault(k, []).extend(frequent_itemsets)

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]["itemset"], search_space[b]["itemset"])
                
                # Second add newset to search_space
                search_space[a][b]["itemset"] = newset
                search_space[a][b]["support"] = self.computeItemsetSupport(newset)
            
            sub_tree = tree[a].copy()
            sub_search_space = search_space[a].copy()  
            
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')

In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [29, 56], [52, 56], [56, 58], [60, 66], [29, 66], [52, 66], [58, 66], [34, 40], [29, 34], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [7, 60], [7, 40], [7, 29], [7, 52], [7, 58], [36, 60], [36, 40], [29, 36], [36, 52], [36, 58], [40, 60], [29, 60], [52, 60], [58, 60], [29, 40], [40, 52], [40, 58], [29, 52], [29, 58], [52, 58]], 3: [[48, 52, 58], [29, 52, 56], [29, 56, 58], [52, 56, 58], [29, 60, 66], [52, 60, 66], [58, 60, 66], [29, 52, 66], [29, 58, 66], [52, 58, 66], [29, 34, 40], [34, 40, 52], [34, 40, 58], [29, 34, 52], [29, 34, 58], [34, 52, 58], [29, 60, 62], [52, 60, 62], [58, 60, 62], [29, 40, 62], [40, 52, 62], [40, 58, 62], [29, 52, 62], [29, 58, 62], [52, 58, 62], [7, 40, 60], [7, 29, 60], [7, 52, 60], [7, 58, 60], [7, 29, 40], [7, 40, 52], [7, 40, 58], [7, 29, 52], [7, 29, 58], [7, 52, 58], [36, 40, 60], [29, 36, 60], [36, 52, 60], [36, 58, 60], [29

### Answer questions here:
**Why don't we compute support of items while reading data?**
- Computing support from the beginning may increase time and computing resources during the data reading process. 

- Calculating the support of items during the initialization of the TP class is a way to sort items according to increasing support. If we just read the data directly, compute support and then arrange them, it will be relatively complicated and resource-consuming. Additionally, separating data reading and TP layer initialization makes the code more readable and maintainable, and provides flexibility for algorithm implementation.

**why should we do sort**
- In Tree Projection algorithm, constructing candidate items based on items with higher support can help reduce the number of items to be examined during the generation of candidate items. By focusing on items with higher support, we can skip over items with lower likelihood of becoming candidate items.

- Sorting items in ascending order of support can also optimize the data traversal process. By traversing items in this order, we can early eliminate items that do not meet the minimum support threshold without the need for further examination. This helps save time and computational resources, improving the algorithm's efficiency.

- Furthermore, minimizing the use of temporary memory resources is another significant advantage of sorting items in ascending order of support. This allows for more efficient management of memory resources and avoids memory overload situations.


**study about python set and its advantages ?**
- Python set ensure that each element appears only once. This will help ensure data integrity by ensuring uniqueness of element sets
- By using an internal hash table, Python set could determine whether a set of elements is a subset of a transaction in the database or not can be done quickly.
- Storing sets of elements as Python set allows the use of set operations such as intersection, union, and complement. These operations can be useful for many data mining tasks, helping to identify frequent sets of elements and association rules.

- In short, Python set provide a flexible and efficient way to work with collections of unique elements, provide fast existence checking, and are compatible with set operations. consistency and integrity of data. These benefits contribute to the performance and reliability of mining algorithms.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- By using the markup mechanism, we retain the original data structure of search_space and tree. This means that these data structures still contain information about all the items and the relationships between them, but only mark those items that have been removed.

- When we need to check or perform other operations on search_space and tree, we can easily identify purged items by checking the pruned status.

- Instead of deleting elements from search_space and tree, marking only requires changing a boolean value. This saves time and resources compared to deleting and restructuring data structures.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

- **Apriori algorithm:**
    - The apriori algorithm uses the method of generating all candidate sets and testing the popularity of candidate sets to find common sets.
    - This algorithm often uses simple data structures such as tables to store candidate sets and frequent itemsets.
    - Apriori algorithms have a performance disadvantage when working with large data sets, this is because it have to generate a very large number of candidate sets and test them. However, this algorithm still works well and is quite easy to implement.

- **Tree Projection:**
    - Tree projection does not generate candidate sets, it relies on the tree structure to eliminate uncommon subsets and only focuses on finding potential branches.
    - This algorithm uses tree structures or similar graphs to store the structure of data and find information about frequent itemsets.
    - Tree Projection can overcome the shortcomings of Apriori when working with large data sets because it does not need to generate candidate sets, it can be said that this algorithm has better performance than Apriori.



# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

### **Brief about the idea:**
- I'm gonna use `Tree Projection` to find frequent itemset in the subdataset which **Churn is True** and the subdataset which **Churn is False**.

    - Firstly, we should discretize the numerical attributes and standardize it to facilitate analysis.

    - Then, we will split the dataset into 2 subsets based on the `Churn` value and transform it into transaction data.

    - Finally, use Tree Projection to find frequent itemsets in both datasets.

In [6]:
# TODO
# YOUR CODE HERE
import pandas as pd
from collections import Counter
from sklearn.preprocessing import KBinsDiscretizer

def transform_to_flat(input_df, frequent_items=None):
    flat_dict = {}
    for idx, row in input_df.iterrows():
        transaction_dict = {}
        for col, value in row.iloc[:-1].items():  # remove last col
            transaction_dict[f"{col} : {value}"] = 1  
        if frequent_items:
            transaction_dict.update(frequent_items.get(idx, {}))
        flat_dict[idx] = transaction_dict
    return flat_dict

def compute_frequencies(data):
    s = Counter()
    for each in data.values():
        s.update(each)
    return dict(s)

### Read data from file

In [7]:
input_file = 'churn.txt'
df = pd.read_csv(input_file)

### Preprocessing - Exploring

In [8]:
# retrieve the index
index = df.index
# create a Pandas Series indicating whether each index is duplicated or not
deDupSeries = index.duplicated(keep='first')
# calculate the number of duplicated rows
num_duplicated_rows = deDupSeries.sum()

if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated line!")
else:
    ext = "lines" if num_duplicated_rows > 1 else "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext)

Raw data have no duplicated line!


In [9]:
num_col_info_df = df.select_dtypes(exclude=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100)

def median(df):
    return (df.quantile(0.5))

def lower_quartile(df):
    return (df.quantile(0.25))

def upper_quartile(df):
    return (df.quantile(0.75))

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0
min,1.0,408.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,23.2,33.0,1.04,0.0,0.0,0.00,0.0
lower_quartile,74.0,408.0,0.0,143.7,87.0,24.43,166.6,87.0,14.16,167.0,87.0,7.52,8.5,3.0,2.30,1.0
median,101.0,415.0,0.0,179.4,101.0,30.50,201.4,100.0,17.12,201.2,100.0,9.05,10.3,4.0,2.78,1.0
upper_quartile,127.0,510.0,20.0,216.4,114.0,36.79,235.3,114.0,20.00,235.3,113.0,10.59,12.1,6.0,3.27,2.0
max,243.0,510.0,51.0,350.8,165.0,59.64,363.7,170.0,30.91,395.0,175.0,17.77,20.0,20.0,5.40,9.0


In [10]:
cat_col_info_df = df.select_dtypes(include=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100)

def num_values(s):
    s = s.astype('str').str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.astype('str').str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,State,Phone,Int'l Plan,VMail Plan,Churn?
missing_ratio,0.0,0.0,0.0,0.0,0.0
num_values,51,3333,2,2,2
value_ratios,"{'WV': 3.2, 'MN': 2.5, 'NY': 2.5, 'AL': 2.4, '...","{'382-4657': 0.0, '348-7071': 0.0, '389-6082':...","{'no': 90.3, 'yes': 9.7}","{'no': 72.3, 'yes': 27.7}","{'False.': 85.5, 'True.': 14.5}"


- **I assumed that `Phone` columns is meaningless in this situation, so i'll drop it.**

- **Although the `VMail Plan` and `VMail Message` columns are two attributes, they have related meanings. If `VMail Plan` = no, then `VMail Message` = 0 and if `VMail Plan` is yes, then `VMail Message` > 0, so I will drop the column `VMail Plan`.**

In [11]:
df.drop(columns=['Phone','VMail Plan'],inplace=True)

- **Okay then, our data have no missing values as well as duplicate rows. Now, I will discrete and normalized the numerical columns.**

In [12]:
# we gonna discrete it into 3 bins, as 0,1,2 represent for low, medium and high.
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df_discretized = pd.DataFrame(discretizer.fit_transform(df[['Account Length', 'Day Mins', 'Day Calls', 'Eve Mins', 'Eve Calls', 'Night Mins', 'Night Calls', 'Intl Mins', 'Intl Calls', 'CustServ Calls']]),
                               columns=['Account Length', 'Day Mins', 'Day Calls', 'Eve Mins', 'Eve Calls', 'Night Mins', 'Night Calls', 'Intl Mins', 'Intl Calls', 'CustServ Calls'])

# concatenate it into original df
df[['Account Length', 'Day Mins', 'Day Calls', 'Eve Mins', 'Eve Calls', 'Night Mins', 'Night Calls', 'Intl Mins', 'Intl Calls', 'CustServ Calls']] = df_discretized
df

,State,Account Length,Area Code,Int'l Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,2.0,415,no,25,2.0,2.0,45.07,1.0,1.0,16.78,2.0,0.0,11.01,1.0,1.0,2.70,1.0,False.
1,OH,1.0,415,no,26,1.0,2.0,27.47,1.0,1.0,16.62,2.0,1.0,11.45,2.0,1.0,3.70,1.0,False.
2,NJ,2.0,415,no,0,2.0,2.0,41.38,0.0,2.0,10.30,0.0,1.0,7.32,2.0,2.0,3.29,0.0,False.
3,OH,1.0,408,yes,0,2.0,0.0,50.90,0.0,0.0,5.26,1.0,0.0,8.86,0.0,2.0,1.78,2.0,False.
4,OK,0.0,415,yes,0,1.0,2.0,28.34,0.0,2.0,12.61,1.0,2.0,8.41,1.0,1.0,2.73,2.0,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,2.0,415,no,36,1.0,0.0,26.55,1.0,2.0,18.32,2.0,0.0,12.56,1.0,2.0,2.67,2.0,False.
3329,WV,0.0,415,no,0,2.0,0.0,39.29,0.0,0.0,13.04,1.0,2.0,8.61,1.0,1.0,2.59,2.0,False.
3330,RI,0.0,510,no,0,1.0,2.0,30.74,2.0,0.0,24.55,1.0,0.0,8.64,2.0,2.0,3.81,2.0,False.
3331,CT,2.0,510,yes,0,2.0,1.0,36.35,0.0,0.0,13.57,0.0,2.0,6.26,0.0,2.0,1.35,2.0,False.


### Split into 2 subdataset and transform into flat file

In [13]:
# split into 2 subdataset and transform into flat file
transaction_churn_true = transform_to_flat(df[df['Churn?'] == 'True.'])
transaction_churn_false = transform_to_flat(df[df['Churn?'] == 'False.'])

# compute frequencies
freq_true = compute_frequencies(transaction_churn_true)
freq_false = compute_frequencies(transaction_churn_false)

### Using Tree Project to find Frequent Itemsets

In [14]:
threshold = 0.35 # 35% of the dataset

In [15]:
true = TP(data=transaction_churn_true,s=freq_true,minSup=len(transaction_churn_true)*threshold)
for (level, sub_list) in true.miningResults().items():
    for each in sub_list:
        print(f"{level} - {each}")

1 - ['Eve Calls : 2.0']
1 - ['Intl Calls : 1.0']
1 - ['Day Calls : 2.0']
1 - ['Intl Mins : 2.0']
1 - ['Eve Mins : 2.0']
1 - ['Area Code : 415']
1 - ['Day Mins : 2.0']
1 - ['CustServ Calls : 2.0']
1 - ["Int'l Plan : no"]
1 - ['VMail Message : 0']
2 - ['Eve Mins : 2.0', 'VMail Message : 0']
2 - ['Area Code : 415', "Int'l Plan : no"]
2 - ['Area Code : 415', 'VMail Message : 0']
2 - ['Day Mins : 2.0', "Int'l Plan : no"]
2 - ['Day Mins : 2.0', 'VMail Message : 0']
2 - ['CustServ Calls : 2.0', "Int'l Plan : no"]
2 - ['CustServ Calls : 2.0', 'VMail Message : 0']
2 - ["Int'l Plan : no", 'VMail Message : 0']
3 - ['Day Mins : 2.0', "Int'l Plan : no", 'VMail Message : 0']
3 - ['CustServ Calls : 2.0', "Int'l Plan : no", 'VMail Message : 0']


In [16]:
false = TP(data=transaction_churn_false,s=freq_false,minSup=len(transaction_churn_false)*threshold)
for (level, sub_list) in false.miningResults().items():
    for each in sub_list:
        print(f"{level} - {each}")

1 - ['Day Mins : 1.0']
1 - ['CustServ Calls : 1.0']
1 - ['Intl Calls : 1.0']
1 - ['CustServ Calls : 2.0']
1 - ['Intl Calls : 2.0']
1 - ['Area Code : 415']
1 - ['VMail Message : 0']
1 - ["Int'l Plan : no"]
2 - ["Int'l Plan : no", 'Intl Calls : 1.0']
2 - ['CustServ Calls : 2.0', "Int'l Plan : no"]
2 - ["Int'l Plan : no", 'Intl Calls : 2.0']
2 - ['Area Code : 415', "Int'l Plan : no"]
2 - ["Int'l Plan : no", 'VMail Message : 0']


### Comment

>**Churn is True**

- Customers exhibit **high communication needs**, as indicated by the high duration and number of morning, evening, and international calls.

- Despite the **high international call** duration, customers **do not sign up** for an international service package. This suggests potential dissatisfaction with the international service offering.

- The presence of **high customer service call** frequency may reflect dissatisfaction with the service quality or other aspects of the telecommunications service.

>**Churn is False**

- There is a **tendency** to use services that involve **calls**, especially during the day.

- The **low number of calls to customer service** may indicate they don't have many problems or have a positive attitude toward service.

- **Less** likely to **sign up for an international plan**, suggesting they may not have a need or interest in international features.